In [46]:
import pandas as pd
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from adapt.feature_based import DANN
import warnings
warnings.simplefilter("ignore")

In [47]:
adam = Adam(learning_rate=0.01)
def get_MLP():
    model=Sequential()
    model.add(Dense(64,input_dim=6,activation="relu"))
    model.add(Dense(256,activation="relu",))
    model.add(Dense(256,activation="relu",))
    model.add(Dense(1))
    model.compile(loss="mse",optimizer=adam)
    return model
def get_DANN(Xs,ys,Xt_train):    
    model = DANN(task=get_MLP(),lambda_=0.1, Xt=Xt_train, metrics=["acc"])
    model.fit(Xs, ys, epochs=100, verbose=0)
    return model

In [ ]:
#input data
path_drill_logging = r'drilling.xlsx'
path_bit = r'BitRecord.xlsx'
data = pd.read_csv(path_drill_logging)
bit = pd.read_excel(path_bit)
#data preprocessing
scaler = StandardScaler()
columns_to_standardize = []
scaler.fit(data[columns_to_standardize])

In [ ]:
#MLP model
#bitsize can be '26"','16"','12-1/4"','8-1/2"','6"'
size = '26"'
same_size_bit = bit[bit['Bit']== size]
#select a bit to predict
well = 1
topdepth = 3000
botdepth = 3546
footages = 546
#get the nowear data of the bit
nowear_data = data[(data['Well']==well)&(data['Depth']>topdepth)&(data['Depth']<=topdepth+30)].loc[:,:]
wear_data = data[(data['Well']==well)&(data['Depth']>topdepth+30)&(data['Depth']<=botdepth)].loc[:,:]
#get the left same size data of the bit
#same_size_bit number
bit_num = len(same_size_bit)
allnowear_data = []
for i in range(bit_num):
    well = same_size_bit.iloc[i,1]
    topdepth = same_size_bit.iloc[i,2]
    botdepth = same_size_bit.iloc[i,3]
    footages = same_size_bit.iloc[i,4]
    allnowear_data.append(data[(data['Well']==well)&(data['Depth']>topdepth)&(data['Depth']<=topdepth+30)].loc[:,:])
allnowear_data = pd.concat(allnowear_data)
leftnowear_data = allnowear_data.drop(nowear_data.index)
print('nowear_data number:',len(nowear_data))
print('leftnowear_data number:',len(leftnowear_data))
#Set the nowear bit data as 7; 3 Divide the training set and test set
features = ['Depth','WOB','RPM','TORQUE','FLWpmps','Density','DT','GR']
xt = nowear_data.loc[:,features].values
xt = scaler.transform(xt)
yt = nowear_data.loc[:,['ROP m/hr']].values
Xt_train, Xt_test, yt_train, yt_test = train_test_split(xt, yt, test_size=0.3, random_state=0)
#将left_bit_data和Xt_train合并
Xs = leftnowear_data.loc[:,features].values
Xs = scaler.transform(Xs)
ys = leftnowear_data.loc[:,['ROP m/hr']].values
# Xs = np.concatenate((Xs,Xt_train),axis=0)
# ys = np.concatenate((ys,yt_train),axis=0)
#train model
model = get_MLP()
model.fit(Xs,ys,epochs=100)
#model=get_WANN(Xs,ys,Xt_train,yt_train)
yt_pred=model.predict(Xt_test)
#evaluation
MLP_r2 = r2_score(yt_test,yt_pred)
MLP_mape =mean_absolute_percentage_error(yt_test,yt_pred)
print('R2:',MLP_r2)
print('MAPE:',MLP_mape)
#predict wear data to get the wf
wear_x = wear_data.loc[:,features].values
wear_y = wear_data.loc[:,['ROP m/hr']].values
wear_x = scaler.transform(wear_x)
pred_y = model.predict(wear_x)
wear_data['ROP0'] = pred_y
wear_data['wf']=wear_data['ROP m/hr']/wear_data['ROP0']
#save the data
wear_data.to_csv('Pre_MLP.csv',index=False)

In [ ]:
#DANN model
#bitsize can be '26"','16"','12-1/4"','8-1/2"','6"'
size = '26"'
same_size_bit = bit[bit['Bit']== size]
#select a bit to predict
well = 1
topdepth = 3000
botdepth = 3546
footages = 546
#get the nowear data of the bit
nowear_data = data[(data['Well']==well)&(data['Depth']>topdepth)&(data['Depth']<=topdepth+30)].loc[:,:]
wear_data = data[(data['Well']==well)&(data['Depth']>topdepth+30)&(data['Depth']<=botdepth)].loc[:,:]
#get the left same size data of the bit
#same_size_bit number
bit_num = len(same_size_bit)
allnowear_data = []
for i in range(bit_num):
    well = same_size_bit.iloc[i,1]
    topdepth = same_size_bit.iloc[i,2]
    botdepth = same_size_bit.iloc[i,3]
    footages = same_size_bit.iloc[i,4]
    allnowear_data.append(data[(data['Well']==well)&(data['Depth']>topdepth)&(data['Depth']<=topdepth+30)].loc[:,:])
allnowear_data = pd.concat(allnowear_data)
leftnowear_data = allnowear_data.drop(nowear_data.index)
print('nowear_data number:',len(nowear_data))
print('leftnowear_data number:',len(leftnowear_data))
#Set the nowear bit data as 7; 3 Divide the training set and test set
features = ['Depth','WOB','RPM','TORQUE','FLWpmps','Density','DT','GR']
xt = nowear_data.loc[:,features].values
xt = scaler.transform(xt)
yt = nowear_data.loc[:,['ROP m/hr']].values
Xt_train, Xt_test, yt_train, yt_test = train_test_split(xt, yt, test_size=0.3, random_state=0)
#将left_bit_data和Xt_train合并
Xs = leftnowear_data.loc[:,features].values
Xs = scaler.transform(Xs)
ys = leftnowear_data.loc[:,['ROP m/hr']].values
Xs = np.concatenate((Xs,Xt_train),axis=0)
ys = np.concatenate((ys,yt_train),axis=0)
#train model
model = get_DANN(Xs,ys,Xt_train)
model.fit(Xs,ys,epochs=100)
yt_pred=model.predict(Xt_test)
#evaluation
MLP_r2 = r2_score(yt_test,yt_pred)
MLP_mape =mean_absolute_percentage_error(yt_test,yt_pred)
print('R2:',MLP_r2)
print('MAPE:',MLP_mape)
#predict wear data to get the wf
wear_x = wear_data.loc[:,features].values
wear_y = wear_data.loc[:,['ROP m/hr']].values
wear_x = scaler.transform(wear_x)
pred_y = model.predict(wear_x)
wear_data['ROP0'] = pred_y
wear_data['wf']=wear_data['ROP m/hr']/wear_data['ROP0']
#save the data
wear_data.to_csv('Pre_DANN.csv',index=False)